# Make buoy data file

Standardize buoy data and put it together in a Pandas dataframe.

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from matplotlib.dates import date2num
import numpy as np
import cartopy
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cmocean.cm as cmo
import os
from collections import OrderedDict

In [2]:
base = 'drive_data/'

In [3]:
# read in buoy info
bys = pd.read_csv(base + 'buoys.csv', index_col=0).to_dict('index')

buoys = {'BOLI': {'ll': ([-94.783, 29.342]), 'color': '#689F38', 'vars': ['WaterT [deg C]', 'Depth [m]', 'Salinity']},
         'MIDG': {'ll': ([-94.875, 29.508]),  'color': '#FBC02D', 'vars': ['WaterT [deg C]', 'Depth [m]', 'Salinity']},
         'FISH': {'ll': ([-94.854, 29.670]),  'color': '#F57C00', 'vars': ['WaterT [deg C]', 'Depth [m]', 'Salinity']},
         'TRIN': {'ll': ([-94.746, 29.661]),  'color': '#FFA000', 'vars': ['WaterT [deg C]', 'Depth [m]', 'Salinity']},
         'OLDR': {'ll': ([-94.783, 29.833]), 'color': '#D32F2F', 'vars': ['WaterT [deg C]', 'Depth [m]', 'Salinity']},
         '8771486': {'ll': ([bys['8771486']['lon'], bys['8771486']['lat']]),  'color': '#0097A7',
                     'vars': ['Water Level [m]', 'AirT [deg C]', 'WaterT [deg C]', 'East [m/s]', 'North [m/s]']},
         '8770613': {'ll': ([bys['8770613']['lon'], bys['8770613']['lat']]),  'color': '#E64A19',
                     'vars': ['Water Level [m]', 'AirT [deg C]', 'WaterT [deg C]', 'East [m/s]', 'North [m/s]', 'Salinity']},
         '8771013': {'ll': ([bys['8771013']['lon'], bys['8771013']['lat']]), 'color': '#AFB42B',
                     'vars': ['Water Level [m]', 'AirT [deg C]', 'WaterT [deg C]', 'East [m/s]', 'North [m/s]', 'Salinity']},
         '8771341': {'ll': ([bys['8771341']['lon'], bys['8771341']['lat']]), 'color': '#388E3C',
                     'vars': ['Water Level [m]', 'AirT [deg C]', 'WaterT [deg C]', 'East [m/s]', 'North [m/s]']},
         '8771450': {'ll': ([bys['8771450']['lon'], bys['8771450']['lat']]), 'color': '#00796B',
                     'vars': ['Water Level [m]', 'AirT [deg C]', 'WaterT [deg C]', 'East [m/s]', 'North [m/s]']},
         'B': {'ll': ([bys['B']['lon'], bys['B']['lat']]), 'color': '#303F9F',
                     'vars': ['AirT [deg C]', 'WaterT [deg C]', 'East [m/s]', 'North [m/s]', 'Salinity', 'Along [cm/s]', 'Across [cm/s]']},
         'g06010': {'ll': ([bys['g06010']['lon'], bys['g06010']['lat']]), 'color': '#388E3C',
                     'vars': ['Along [cm/s]', 'Across [cm/s]']},
         '42035': {'ll': ([bys['42035']['lon'], bys['42035']['lat']]), 'color': '#5D4037',
                     'vars': ['AirT [deg C]', 'WaterT [deg C]', 'East [m/s]', 'North [m/s]']},
         '8770808': {'ll': ([bys['8770808']['lon'], bys['8770808']['lat']]), 'color': '#512DA8',
                     'vars': ['Water Level [m]', 'AirT [deg C]', 'WaterT [deg C]', 'East [m/s]', 'North [m/s]']},
         '8770777': {'ll': ([bys['8770777']['lon'], bys['8770777']['lat']]), 'color': '#C2185B',
                     'vars': ['Water Level [m]', 'AirT [deg C]', 'WaterT [deg C]', 'East [m/s]', 'North [m/s]']},
         '8770822': {'ll': ([bys['8770822']['lon'], bys['8770822']['lat']]), 'color': '#455A64',
                     'vars': ['Water Level [m]', 'AirT [deg C]', 'WaterT [deg C]', 'East [m/s]', 'North [m/s]']},
         '8770971': {'ll': ([bys['8770971']['lon'], bys['8770971']['lat']]), 'color': '#7B1FA2',
                     'vars': ['Water Level [m]', 'AirT [deg C]', 'WaterT [deg C]', 'East [m/s]', 'North [m/s]']},
         '8771972': {'ll': ([bys['8771972']['lon'], bys['8771972']['lat']]), 'color': '#0288D1',
                     'vars': ['Water Level [m]', 'AirT [deg C]', 'WaterT [deg C]', 'East [m/s]', 'North [m/s]']}
        }


buoys_ordered = ['8770777', '8770613', 'OLDR', 'FISH', 'TRIN', 'MIDG', '8771013', 'BOLI', 'g06010', '8771341',
                '8771450', '8771486', '8771972', '8770971', '8770808', 'B', '42035', '8770822']
# buoys down main bay
buoys_main = ['8770613', 'FISH', 'TRIN', 'MIDG', '8771013', 'BOLI', 'g06010', '8771341', '8771450']

In [4]:
savename = base + 'data.csv'
dstart = '2017-06-01'; dend = '2017-12-31'
tz = 'US/Central'
if not os.path.exists(savename):
            
    ## River inflow to Trinity Bay ##
    # https://txpub.usgs.gov/txwaterdashboard/index.html
    # direct link:
    url = 'https://nwis.waterdata.usgs.gov/usa/nwis/uv/?cb_00060=on&cb_00065=on&format=rdb&site_no=08067070&period=&begin_date=' \
            + dstart + '&end_date=' + dend
    river = pd.read_table(url, parse_dates=True, skiprows=29, header=None, usecols=[2,4], index_col=0,
                     names=['Dates [CDT]', 'Trinity flow rate [m^3/s]'])[dstart:dend].tz_localize('US/Central', ambiguous=True)
#     river = pd.read_table('../usgs/08067070.txt', parse_dates=True, skiprows=29, header=None, usecols=[2,4], index_col=0,
#                      names=['Dates [CDT]', 'Trinity flow rate [m^3/s]'])[dstart:dend].tz_localize('US/Central', ambiguous=True)
    river = river*0.3048**3  # to m^3/s
    river = river.resample('1H', base=0).mean()
    df = pd.DataFrame(index=river.index, data=river)
    
    ## TWDB ##
#     # Boli can be dealt with separately
#     dft = pd.read_csv('../twdb/Boli.csv', index_col=0, parse_dates=True)[dstart:dend].tz_localize('UTC').tz_convert(tz)
#     dft.drop(['Datetime_(UTC).1', 'water_conductivity_mS/cm',
#               'instrument_battery_voltage', 'water_specific_conductivity_mS/cm'], axis=1, inplace=True)
#     dft.rename(columns={'water_temp_C': 'BOLI: WaterT [deg C]',
#                         'water_depth_m_nonvented': 'BOLI: Depth [m]',
#                         'water_salinity_PSU': 'BOLI: Salinity'}, inplace=True)
#     dft = dft.resample('1H', base=0).mean()
#     df['BOLI: WaterT [deg C]'] = dft['BOLI: WaterT [deg C]']
#     df['BOLI: Depth [m]'] = dft['BOLI: Depth [m]']
#     df['BOLI: Salinity'] = dft['BOLI: Salinity']
    
    names = ['FISH', 'MIDG', 'OLDR', 'TRIN', 'BOLI']
    Files = ['seawater_salinity', 'water_depth_nonvented', 'water_temperature']
    filenames = ['Salinity', 'Depth [m]', 'WaterT [deg C]']
    base = 'https://waterdatafortexas.org/coastal/api/stations/' 
    for name  in names:
        for File, filename  in zip(Files, filenames):
            # read in as UTC
            url = base + name + '/data/' + File + '?output_format=csv&binning=hour'
            dft = pd.read_csv(url, index_col=0,
                             parse_dates=True, comment='#', header=0, names=['dates [local]', filename])[dstart:dend].tz_localize('UTC').tz_convert(tz)
#             dft = pd.read_csv('../twdb/' + name + File + '.csv', index_col=0,
#                              parse_dates=True, comment='#', header=0, names=['dates [local]', filename])[dstart:dend].tz_localize('UTC').tz_convert(tz)
            dft = dft.resample('1H', base=0).mean()
            df[name + ': ' + filename] = dft
    
    ## TABS ##
    for table in ['met', 'salt', 'ven']:
        url = 'http://localhost/tabswebsite/subpages/tabsquery.php?Buoyname=B&table=' + table + '&Datatype=download&units=M&tz=US/Central&model=False&datepicker='
        url += dstart + '+-+' + dend
        dft = pd.read_table(url, parse_dates=True, index_col=0, na_values=-999)[dstart:dend].tz_localize(tz, ambiguous=True)
#         dft = pd.read_table('../buoys/tabs_B_' + table, index_col=0,
#                          parse_dates=True)[dstart:dend].tz_localize(tz, ambiguous=True)
        dft = dft.resample('1H', base=0).mean()
        for var in buoys['B']['vars']:
            if var in dft.columns:
                df['B: ' + var] = dft[var]
        
    
    ## NOS/NDBC buoys ##
    for buoy in buoys:
        if len(buoy) < 5:
            continue
        url = 'http://localhost/tabswebsite/subpages/tabsquery.php?Buoyname=' + buoy + '&Datatype=download&units=M&tz=US/Central&model=False&datepicker='
        url += dstart + '+-+' + dend
        dft = pd.read_table(url, parse_dates=True, index_col=0, na_values=-999)[dstart:dend].tz_localize(tz, ambiguous=True)
#         dft = pd.read_table('../buoys/' + buoy, index_col=0, parse_dates=True, na_values=-999.00)[dstart:dend].tz_localize(tz, ambiguous=True)
        dft = dft.resample('1H', base=0).mean()
        for var in buoys[buoy]['vars']:
            if var in dft.columns:
                df[buoy + ': ' + var] = dft[var]
    
    ## Vertical datum
    # The vertical datum for the bathymetry is mean high water (in file "Galveston_DEM_metadata.html"). 
    # The sea level measurements that I use are in mean sea level, so need to adjust.
    # using station datum information from e.g. https://tidesandcurrents.noaa.gov/datums.html?id=8771013
    df['8771013: Water Level [m]'] -= (1.610 - 1.467)  # MHW - MSL datums, meters
    df['8771341: Water Level [m]'] -= (3.249 - 3.072)  # MHW - MSL datums, meters
    df['8770777: Water Level [m]'] -= (2.140 - 1.951)  # MHW - MSL datums, meters
    df['8770613: Water Level [m]'] -= (1.967 - 1.807)  # MHW - MSL datums, meters
    df['8771450: Water Level [m]'] -= (1.739 - 1.588)  # MHW - MSL datums, meters
    df['8771486: Water Level [m]'] -= (6.311 - 6.183)  # MHW - MSL datums, meters
    df['8771972: Water Level [m]'] -= (1.621 - 1.498)  # MHW - MSL datums, meters
    df['8770971: Water Level [m]'] -= (1.425 - 1.259)  # MHW - MSL datums, meters
    df['8770808: Water Level [m]'] -= (4.462 - 4.299)  # MHW - MSL datums, meters
    df['8770822: Water Level [m]'] -= (1.507 - 1.308)  # MHW - MSL datums, meters

    df.to_csv(savename)